## Conversion workflow
The diagram below illustrations the high-level workflow for converting your model:

![convert](../assets/convert.png)

You can convert your model using one of the following options:
1. [Python API](https://www.tensorflow.org/lite/models/convert/convert_models#python_api) (recommended): This allows you to integrate the conversion into your development pipeline, apply optimizations, add metadata and many other tasks that simplify the conversion process.
2. [Command line](https://www.tensorflow.org/lite/models/convert/convert_models#cmdline): This only supports basic model conversion.

> NOTE: In case you encounter any issues during model conversion, create a [GitHub issue](https://github.com/tensorflow/tensorflow/issues/new?template=60-tflite-converter-issue.md).

## Python API

Helper code: To learn more about the TensorFlow Lite converter API, run print(help(tf.lite.TFLiteConverter)).

使用 轉換 TensorFlow 模型 [tf.lite.TFLiteConverter](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter)。TensorFlow 模型使用 SavedModel 格式存儲，並使用高級tf.keras.*API（Keras 模型）或低級tf.*API（從中生成具體函數）生成。因此，您有以下三個選項（示例位於接下來的幾節中）：

* [tf.lite.TFLiteConverter.from_saved_model()](https://www.tensorflow.org/lite/api_docs/python/tf/lite/TFLiteConverter#from_saved_model)（推薦）：轉換一個SavedModel。
* [tf.lite.TFLiteConverter.from_keras_model()](https://www.tensorflow.org/lite/api_docs/python/tf/lite/TFLiteConverter#from_keras_model)：轉換 Keras模型。
* [tf.lite.TFLiteConverter.from_concrete_functions()](https://www.tensorflow.org/lite/api_docs/python/tf/lite/TFLiteConverter#from_concrete_functions): 轉換 具體函數。

In [3]:
import tensorflow as tf
print(help(tf.lite.TFLiteConverter))

Help on class TFLiteConverterV2 in module tensorflow.lite.python.lite:

class TFLiteConverterV2(TFLiteFrozenGraphConverterV2)
 |  TFLiteConverterV2(funcs, trackable_obj=None)
 |  
 |  Converts a TensorFlow model into TensorFlow Lite model.
 |  
 |  Attributes:
 |    optimizations: Experimental flag, subject to change. Set of optimizations to
 |      apply. e.g {tf.lite.Optimize.DEFAULT}. (default None, must be None or a
 |      set of values of type `tf.lite.Optimize`)
 |    representative_dataset: A generator function used for integer quantization
 |      where each generated sample has the same order, type and shape as the
 |      inputs to the model. Usually, this is a small subset of a few hundred
 |      samples randomly chosen, in no particular order, from the training or
 |      evaluation dataset. This is an optional attribute, but required for full
 |      integer quantization, i.e, if `tf.int8` is the only supported type in
 |      `target_spec.supported_types`. Refer to `tf.

### Convert a SavedModel (recommended) 
The following example shows how to convert a [SavedModel](https://www.tensorflow.org/guide/saved_model) into a TensorFlow Lite model.

In [1]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model_dir") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

OSError: SavedModel file does not exist at: saved_model_dir\{saved_model.pbtxt|saved_model.pb}

### Convert a Keras model
The following example shows how to convert a [Keras](https://www.tensorflow.org/guide/keras/overview) model into a TensorFlow Lite model.

In [7]:
import tensorflow as tf

# Create a model using high-level tf.keras.* APIs
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1]),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)
])
model.compile(optimizer='sgd', loss='mean_squared_error') # compile the model
model.fit(x=[-1, 0, 1], y=[-3, -1, 1], epochs=5) # train the model
# (to generate a SavedModel) tf.saved_model.save(model, "saved_model_keras_dir")

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Epoch 1/5
1/1 [==============================] - 1s 934ms/step - loss: 3.4129
Epoch 2/5
1/1 [==============================] - 0s 6ms/step - loss: 3.2920
Epoch 3/5
1/1 [==============================] - 0s 6ms/step - loss: 3.1562
Epoch 4/5
1/1 [==============================] - 0s 6ms/step - loss: 3.0246
Epoch 5/5
1/1 [==============================] - 0s 6ms/step - loss: 2.8965
INFO:tensorflow:Assets written to: C:\Users\chiou\AppData\Local\Temp\tmp3u_7o_lv\assets


### Convert concrete functions
The following example shows how to convert [concrete functions](https://www.tensorflow.org/guide/intro_to_graphs) into a TensorFlow Lite model.

In [4]:
import tensorflow as tf

# Create a model using low-level tf.* APIs
class Squared(tf.Module):
  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float32)])
  def __call__(self, x):
    return tf.square(x)
model = Squared()
# (ro run your model) result = Squared(5.0) # This prints "25.0"
# (to generate a SavedModel) tf.saved_model.save(model, "saved_model_tf_dir")
concrete_func = model.__call__.get_concrete_function()

# Convert the model.

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func],
                                                            model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\chiou\AppData\Local\Temp\tmpttcokrup\assets


## Other features
* Apply [optimizations](https://www.tensorflow.org/lite/performance/model_optimization). A common optimization used is [post training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization), which can further reduce your model latency and size with minimal loss in accuracy.
* Add [metadata](https://www.tensorflow.org/lite/models/convert/metadata), which makes it easier to create platform specific wrapper code when deploying models on devices.

## Conversion errors

如果想了解轉換錯誤以及解決方法，可以參考[轉換錯誤](https://www.tensorflow.org/lite/models/convert/convert_models#conversion_errors)

**1**
**錯誤**：Some ops are not supported by the native TFLite runtime, you can enable TF kernels fallback using TF Select. See instructions: <a href="https://www.tensorflow.org/lite/guide/ops_select">https://www.tensorflow.org/lite/guide/ops_select</a> TF Select ops: ..., .., ...

**解決方案**：出現此錯誤是因為您的模型具有沒有相應 TFLite 實現的 TF 操作。您可以通過 [使用 TFLite 模型中的 TF 操作](https://www.tensorflow.org/lite/guide/ops_select)來解決此問題 （推薦）。如[果您只想生成帶有 TFLite 操作的模型，您可以在Github 問題 #21526](https://github.com/tensorflow/tensorflow/issues/21526)中添加對缺少的 TFLite 操作的請求 （如果您的請求尚未被提及，請發表評論）或 自己[創建 TFLite 操作](https://www.tensorflow.org/lite/guide/ops_custom#create_and_register_the_operator)。

**2**
錯誤：.. is neither a custom op nor a flex op
解決方案：如果這個 TF op 是：
* Supported in TF: The error occurs because the TF op is missing from the [allowlist](https://www.tensorflow.org/lite/guide/op_select_allowlist) (an exhaustive list of TF ops supported by TFLite). You can resolve this as follows:
* 1. [Add missing ops to the allowlist.](https://www.tensorflow.org/lite/guide/op_select_allowlist#add_tensorflow_core_operators_to_the_allowed_list)
* 2. [Convert the TF model to a TFLite model and run inference.](https://www.tensorflow.org/lite/guide/ops_select)


# Select TensorFlow operators
## Convert a model
The following example shows how to generate a TensorFlow Lite model with select TensorFlow ops.

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)